<a href="https://colab.research.google.com/github/khassibi/fourier-neural-operator/blob/main/Pandemaniac_2_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [144]:
# importing the module
import json
import numpy as np
import networkx as nx
import random
import pickle

In [145]:
# getting the data
from google.colab import drive
drive.mount('/content/drive/')
%cd drive/MyDrive
# to use graph data, copy sample_graphs linked below and add a shortcut to your "MyDrive" folder
# https://drive.google.com/drive/u/0/folders/1AHIrdyHnmcbxb0rjtTJPrK6-fEwmlZdQ

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive'
/content/drive/MyDrive


# Generating Graphs

## Making Web Graph

In [ ]:
with open('sample_graphs/web_final.pickle', 'rb') as handle:
    web = pickle.load(handle)
web_graph=nx.Graph()

## Making Preferential Attachment Graph

In [ ]:
def generate_pref_network(T):
  """
  T is the number of nodes
  Return an undirected networkxgraph of the residents and a list of the 
  degrees of the nodes in the network
  """
  G = nx.Graph()
  G.add_node(0)
  G.add_node(1)
  G.add_edge(0,1)
  # For each node starting at 2, connect it probabilistically to a pre-existing
  # node
  for i in range(2, T):
      prob = np.random.uniform()
      cum_degs = np.cumsum(np.array(G.degree())[:,1])
      prob_bins = cum_degs / cum_degs[-1]
      for j in range(0, i):
        if prob <= prob_bins[j]:
          G.add_node(i)
          G.add_edge(i,j)
          break

  return G

## Making SSBM

In [ ]:
def sbm(n, k, A, B):
    
    G = nx.Graph()
    
    which_com = {}
    
    for i in range(n):
        G.add_node(i)
        which_com[i] = np.floor(np.random.uniform()*k)
    
    for i in range(n): # for all pairs of nodes
        for j in range(i+1, n):
            if which_com[i] == which_com[j]: # if same community
                if np.random.uniform() < A:  # add an edge w.p. A
                    G.add_edge(i, j,  weight = 1)
            else:                            # otherwise
                if np.random.uniform() < B:  # add an edge w.p. B
                    G.add_edge(i, j,  weight = 1)
    return G

## Making Erdo-Renyi

In [ ]:
def erdos_renyi(n, p):
    G = nx.Graph()
    for i in range(n):
        G.add_node(i)
    for i in range(n):
        for j in range(i+1, n):
            if np.random.uniform() < p:
                G.add_edge(i, j)
    return G

## Getting the graph from JSON file

In [ ]:
def from_json(name):
  # Opening JSON file
  data = {}
  with open('sample_graphs/' + graph_name + '.json') as json_file:
      data = json.load(json_file)
  
      # Print the type of data variable
      print("Type:", type(data))
      
  # convert json dictionary to integer
  def make_dict_int(x):
      return {int(k):list(np.array(v).astype(int)) for k,v in x.items()}
  data = make_dict_int(data)
  G = nx.Graph()
  G = nx.from_dict_of_lists(data)
  return G

# Our algorithms

## Friend of Friends Algorithm

In [146]:
"""
First attempt: End strategy
"""
"""

node_list = G.nodes()
random_nodes50 = random.sample(node_list, 50)
friends = []

for node in random_nodes50:

  friends = list(G.neighbors(node))
  fof = random.choice(friends)

  if fof not in friends:
    friends.append(fof)

"""

def friend_of_friend(G, num_seeds):
  """ Friend of a friend a while loop """
  node_list = list(G.nodes())
  friends = []

  while len(friends) != num_seeds:
    node = random.choice(node_list)
    friends_of_nodes = list(G.neighbors(node))

    if len(friends_of_nodes) != 0:
      node_friend = random.choice(friends_of_nodes)
    else:
      continue

    if node_friend not in friends:
      friends.append(node_friend)

  return friends

## Random 50 Strategy 

In [ ]:
def random50(G, num_seeds):
  # submission for a day we are tired
  node_list = G.nodes()
  random_nodes50 = random.sample(node_list, num_seeds)
  return random_nodes50


# Testing the Code

In [147]:
## Pick a graph
num_seeds = 10
graph_name = 'J.10.30'
G = from_json(graph_name)

## Pick an algorithm
algorithm_name = 'Friend of Friend'

seed = []
for i in range(50):
  ## Create seed
  seed += friend_of_friend(G, num_seeds)

## Name the submission
submission_name = algorithm_name + ' on ' + graph_name
with open('sample_graphs/submissions/' + submission_name + '.txt', 'w+') as f:
    content = '\n'.join(str(x) for x in seed)
    f.write(content)

Type: <class 'dict'>
